# Experiments with Bayesian Machine Learning Models

In this section, we build several multi-class classification models using Bayesian Machine Learning for the CiteSeer Dataset.

1. Model 4a: A model with one-hot encoded vectors as the input features
2. Model 4b: A model with Topics learned from Latent Semantic Analysis as the input features
3. Model 4c: A model with Topics learned from Latent Dirichlet Allocation as the input features
   
<!-- 3. ???
4. Model xyz: A model with Tensorflow embeddings for the vectors

5. Model xyz: The effect of priors on prediction with the best model till now.
6. Model xyz: Changing model architecture
 -->

## Brief Intro to Bayesian Machine Learning

Bayesian Machine Learning models are built on the principles of Bayesian statistics. In these models, we assume prior distributions of the parameters, representing our initial beliefs/knowledge about the parameters before seeing any data. As we train the model and observe data, we **update the priors using Bayes' theorem to obtain posterior distributions**. The posterior distributions represent our updated beliefs about the parameters after incorporating the data.

A point to note is that Bayesian Machine Learning can be mathematically challenging due to the need to compute integrals over complex, high-dimensional probability distributions. These integrals are often intractable, making it difficult to directly calculate the posterior distributions of the model parameters. This is where Markov Chain Monte Carlo (MCMC) methods come to the rescue. MCMC algorithms, such as the No-U-Turn Sampler (NUTS) and Hamiltonian Monte Carlo (HMC), allow us to approximate these integrals by generating samples from the posterior distribution. Using these samples, we can estimate the posterior distributions and make inferences about the model parameters, effectively bypassing the need for exact integration -- the essence of any Monte Carlo method. This makes MCMC a powerful tool for Bayesian inference, enabling us to tackle complex problems that would otherwise be computationally prohibitive. We are going to use a popular implementation in Python -- PyMC -- to build our models.

A key advantage of Bayesian ML is that it provides uncertainty estimates directly. That is, for any prediction, we can quantify the confidence in that prediction by looking at the spread of the posterior distribution. This is particularly useful in applications where understanding the uncertainty is crucial, such as in medical diagnosis, financial forecasting, autonomous driving, or modeling human behavior.

In contrast, traditional Deep Learning models typically do not provide uncertainty estimates directly. These models are often deterministic, meaning that for a given input, they produce a single output without any measure of uncertainty. To estimate uncertainty in Deep Learning models, additional techniques are required, such as:

1. Dropout: A regularization technique where random neurons are dropped during training. At inference time, dropout can be used to create an ensemble of models, and the variance in their predictions can be used as an uncertainty estimate.

2. Ensemble Methods: Training multiple models independently and combining their predictions. The variance in the predictions of the ensemble members can be used to estimate uncertainty.

While these methods can provide some measure of uncertainty, they are often less straightforward and may not capture the full range of uncertainty as effectively as Bayesian methods.

In summary, Bayesian Machine Learning models offer a more natural and direct way to quantify uncertainty in predictions. This can be a significant advantage over other Machine Learning and Deep Learning methods in many applications.

A big thanks for this section goes to Michael J. Schoelles for his awesome course on Cognitive Modeling at Rensselaer Polytechnic Institute. This course and a related course -- Programming for Cognitive Science and AI -- by Dr. Schoelles built the foundation of my Machine Learning knowledge and helped me immensely in developing this tutorial.  

## How to install PyMC?

To me, the biggest challenge in doing Bayesian Machine Learning is setting up the packages correctly- specifically, the pymc/pymc3 packages and their many, many dependencies. It used to be very painful, but now things seem to have improved with the migration from "pymc3" to "pymc". Here, I will discuss the minimum steps I needed to get going in January 2025. Please see the [full installation guide here](https://www.pymc.io/projects/docs/en/latest/installation.html). Finally, I am using a Windows machine, but the installation seemed a bit easier on Linux machines.

Basically, all we need to do is create and activate a virtual environment. Please DO NOT ignore the virtual env step. Things related to pymc3 broke my Python installation several times (yeah, several times, as I needed to be sure PyMC was breaking it. Sigh).

We will be using **Conda** virtual environments. We will follow the first two steps from the install guide.

       conda create -c conda-forge -n pymc_env "pymc>=5"
       conda activate pymc_env

Now, everything should be working nicely now! 

Of course, I tried other ways of doing it, but with limited success. The above method did seem to work the best. I am still getting the "BLAS" error mentioning C/Numpy and whatnot -- perhaps following the later optional parts from the install guide will help. But the speed of running the models seems good, and everything works, so I will address the error later to see if things improve more. Hit me up if you run into any problems or if you find a better solution.

<u>OLD but keeping it in case handy</u>: Previously, we had to install some toolchain-related and other necessary packages. Seems not necessary to do now.

       conda install mkl-service libpython m2w64-toolchain


## Get necessary things and create necessary functions

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Functions to evaluate: accuracy, precision, recall, f1-score
    # Many options. We will do it as we please, sometimes on our own
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,roc_auc_score,roc_curve

# We need this one to avoid tensor issues in pymc. We will see how you can utilize this.
import pytensor.tensor as pt
import pymc as pm
import arviz as az # Many things, one of which is saving the traces
import pickle # To save models -- THIS FIRST CHOICE DID NOT WORK. So, using "dill" which works roughly the same way as pickle
import dill as dpickle # To save models -- The second choice that worked.
SAVE_DIR = "Saved_BL_models_Exp4"

# For plotting and summarizing pymc things conveniently
from arviz import plot_posterior, plot_trace, plot_pair, plot_kde, plot_forest
from arviz import summary, hdi


WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
# Four measures of model goodness together
def calculate_results(y_true, y_pred):
# Calculate model accuracy
# Returns a dictionary of accuracy, precision, recall, f1-score.
  
# y_true: true labels in the form of a 1D array
# y_pred: predicted labels in the form of a 1D array
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall and f1 score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
    return(model_results)

In [3]:
# The metrics here are calculated from the confusion matrix
def accuracy(cm):
    return np.trace(cm)/np.sum(cm)

def error_rate(cm):
    return (cm[0,1]+cm[1,0])/np.sum(cm)

def precision(cm):
    return cm[1,1]/(cm[1,1]+cm[0,1])

def recall(cm):
    return cm[1,1]/(cm[1,1] + cm[1,0])

def F1(cm):
    tp,fp,fn = cm[1,1],cm[0,1],cm[1,0]
    return tp/(tp + 1/2*(fp+fn))

## Get the Dataset

We use the CiteSeer Node Classification task for our experiments. This dataset is one of the Planetoid datasets from the torch_geometric.datasets.

In [4]:
# Import dataset from PyTorch Geometric
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root=".", name="CiteSeer")

data = dataset[0]

# Print information about the dataset
print("Dataset name:", dataset)
print("Input Text Data shape and type:", data.x.shape, data.x.dtype)
print("First five rows of the text data:\n", data.x[0:5, :])

Dataset name: CiteSeer()
Input Text Data shape and type: torch.Size([3327, 3703]) torch.float32
First five rows of the text data:
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


As we see, the dataset has 3327 documents as rows, made up of 3703 unique words. The documents are represented as one-hot vectors of length 3703. One hot vectors simply mean that if a word exists, then we assign it's magnitude to be 1 and if not, then we assign the magnitude to be 0. We just to need to follow the same order of words for each document, and that is it.

An interesting point is the array type, which is "torch.tensor". Torch tensors are perfectly compatible with Numpy, so we should be fine.

**Important**: Please note that we are not using all of the data available, rather using only about half of the documents. Moreover, we are using just 120 documents for training. The reason is that these are stipulations imposed in benchmarking different models that we saw earlier. We keep the split as is to be able to compare with the state-of-the-art results.

Now, we are ready to get modeling!

### Train-Validation-Test Set Split

In [5]:
train_sentences_one_hot = data.x[data.train_mask]
train_labels = data.y[data.train_mask]

val_sentences_one_hot = data.x[data.val_mask]
val_labels = data.y[data.val_mask]

test_sentences_one_hot = data.x[data.test_mask]
test_labels = data.y[data.test_mask]

print(train_sentences_one_hot.shape, val_sentences_one_hot.shape, test_sentences_one_hot.shape)

torch.Size([120, 3703]) torch.Size([500, 3703]) torch.Size([1000, 3703])


# Model Set 4: Bayesian Learning Models

## Model 4a: Using all 3703 words directly as features for classification

In [6]:
num_classes = np.unique(data.y).shape[0]
num_features = data.x.shape[1]

np.array(train_sentences_one_hot)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [7]:
# %%timeit

# with pm.Model() as model_4a:
#     alpha = pm.Normal('alpha', mu=0, sigma=2, shape= num_classes)
#     beta = pm.Normal('beta', mu=0, sigma=2, shape=(num_features, num_classes))

#     mu = alpha + pm.math.dot(np.array(train_sentences_one_hot), beta) # The observed input here!
#     # theta = tt.nnet.softmax(mu)
#     theta = pt.special.softmax(mu, axis = 1) # Softmax over the mu values
#     yl = pm.Categorical('yl', p=theta, observed = train_labels) # observed train_labels here
#     trace_3a = pm.sample(2000) 
# plot_trace(trace_3a)
# summary(trace_3a)

### What did we learn from this experiment?
The number of features is a big challenge for us. The processes taking too long -- one sampling process did not end in three hours. To have a chance at completing the computations on our personal computers, we need to find a way to reduce the complexity.

## Model 4b: LSA Topic Model + PyMC model

We will use a "Topic modeling" approach as a form of feature engineering. Specifically, we will reduce the many words down to some topics that explain most of our data. This way, these topics would be our features, reducing the number of features.

First, we will use a Latent Semantic Analysis or Indexing (LSA/LSI), a popular topic modeling approach. LSA combines a process of TF-IDF vectorization (discussed in Section 1) with a Singular Value Decomposition (SVD) with truncation (simply, Truncated SVD). It is the SVD step that gives the topics. The SVD process decomposes or factorizes our document-word matrix (A) into three factor matrices: A document-topic matrix ($U$), a diagonal matrix containing the singular values ($S$), and a topic-word matrix ($V^T$). 

\begin{equation}
A = U*S*V^T
\end{equation}

Essentially, we are transforming our original data vectors into a latent space of eigenvectors. These eigenvectors represent our topics of interest. The Singular values -- which are the square roots of the eigenvalues corresponding to the eigenvectors -- indicate how useful the eigenvectors are in explaining the texts. When we truncate, we keep only the highest n number of singular values and their corresponding eigenvectors and ignore the rest. In a way, we look for a more compact representation of our data in latent space using a only few of the eigenvalues learned through SVD.

**Why not PCA?** Another choice is using Principal Component Analysis or PCA, but it works better with dense data. For sparse data like the word vectors we have, SVD is more appropriate. Notably, PCA is a special case of SVD when it is performed on the covariance matrix. SVD applies to any matrix, especially when centering and calculating variance based on that does not make much sense.

Another option we will use later is Latent Dirichlet Allocation, another topic modeling technique which is based on the Bayes theorem.

In [8]:
from sklearn.pipeline import Pipeline

# Vector representations and transformations
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD, NMF

# Classifiers assemble!
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

### Latent Semantic Analysis (LSA)

In [9]:
# Do LSA!
num_topics = 120 # Max_topics = min (total number of features, total number of documents for training)
lsa_topic_model_4b = Pipeline([
    ("tfidf", TfidfTransformer()),
    ("svd", TruncatedSVD(n_components = num_topics))])
lsa_topic_model_4b.fit(train_sentences_one_hot, train_labels)

# lsa_topic_model_4b.transform(train_sentences_one_hot)

# print(calculate_results(y_pred = model_4b.predict(train_sentences_one_hot), y_true = train_labels))


Pipeline(steps=[('tfidf', TfidfTransformer()),
                ('svd', TruncatedSVD(n_components=120))])

### PyMC model

In [10]:
num_classes = np.unique(data.y).shape[0]
num_features = num_topics ## An important change!

# Input data

train_X = lsa_topic_model_4b.transform(train_sentences_one_hot)
train_y = train_labels

# Define and fit model.
with pm.Model() as bayes_model_4b:
    X = pm.Data("X", train_X)
    y = pm.Data("y", train_y)
    
    alpha = pm.Normal('alpha', mu=0, sigma=2, shape= num_classes) # The intercepts of linear regression
    beta = pm.Normal('beta', mu=0, sigma=2, shape=(num_features, num_classes)) # The coefficients

    mu = alpha + pm.math.dot(X, beta) # The observed input here!
        # NOTE: We wrapped our input data with a name. 
        # This one is needed for predicting on new data (validation, testing)
        # Without it, the line would look like this:
    # mu = alpha + pm.math.dot(train_X, beta) # The observed input here!
    
    theta = pt.special.softmax(mu, axis = 1) # Softmax probability over the mu values

    yl = pm.Categorical('yl', p=theta, observed = y, shape = X.shape[0]) # observed train_labels here

    trace_4b = pm.sample(2000)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta]


Output()

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 29 seconds.


In [11]:
# plot_trace(trace_4b)
summary(trace_4b)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],0.058,0.947,-1.734,1.790,0.017,0.012,3129.0,4777.0,1.0
alpha[1],-0.025,0.940,-1.722,1.762,0.017,0.012,3072.0,4700.0,1.0
alpha[2],0.053,0.943,-1.874,1.746,0.017,0.012,3039.0,4723.0,1.0
alpha[3],-0.006,0.959,-1.787,1.813,0.017,0.012,3184.0,4707.0,1.0
alpha[4],0.018,0.963,-1.827,1.794,0.017,0.012,3116.0,4867.0,1.0
...,...,...,...,...,...,...,...,...,...
"beta[119, 1]",-0.088,1.968,-3.818,3.586,0.016,0.027,14980.0,5353.0,1.0
"beta[119, 2]",-0.006,1.951,-3.589,3.841,0.017,0.026,13906.0,5448.0,1.0
"beta[119, 3]",0.014,1.942,-3.482,3.731,0.016,0.028,14625.0,5298.0,1.0
"beta[119, 4]",-0.012,1.919,-3.542,3.647,0.017,0.024,12322.0,5971.0,1.0


### Predict using the Posterior Distribution

In [12]:
# Posterior predictive sampling 
posterior_predictive_4b = pm.sample_posterior_predictive(trace_4b, predictions = 1, model = bayes_model_4b)

Sampling: [yl]


Output()

In [13]:
posterior_predictive_4b["predictions"]["yl"].shape

(4, 2000, 120)

**Notes about traces and the sample_posterior_predictive**:
We ran four chains, and in each chain, we sampled 2000 times. For each of $4\times2000 = 8000$ samples, we have a prediction. Let us see how the prediction accuracy changed with learning.

### Evaluating Accuracy

First, let's check with a single simulation or a set of predictions.

In [14]:
y_pred = posterior_predictive_4b["predictions"]["yl"][3][0]
y_true = train_labels
calculate_results(y_pred = y_pred, y_true = y_true) 

{'accuracy': 51.66666666666667,
 'precision': 0.5467220602153137,
 'recall': 0.5166666666666667,
 'f1': 0.5207835123598934}

The above function can do only one comparison at a time, although gives us all four measures together. Let's break them up. 

In [15]:
y_pred_all = posterior_predictive_4b["predictions"]["yl"]

# The first chain only
cm = [confusion_matrix(y_true = train_labels, y_pred = y_pred_all[0][i]) for i in range(y_pred_all[0].shape[0])]

In [16]:
accuracy_ = [accuracy(item) for item in cm]
pd.DataFrame({'accuracy':accuracy_}).describe()

,accuracy
count,2000.000000
mean,0.513808
std,0.046658
min,0.350000
25%,0.483333
50%,0.516667
75%,0.541667
max,0.691667


**Notes on Accuracy**: As we see, the average accuracy is about 50% which is three times better than random. At best, we got about 65% correct; at worst, we got about 33% correct. However, the training accuracy is much lower than any of the models we built so far and close to the test accuracies of the Shallow ML methods (Section 1) and Deep Learning models (Section 2), but worse than the GNNs (Section 3) that used additional information of the edges.

1. The role of priors. If our priors are too far off the real distributions, we need a lot of data for the posterior distributions to closely resemble the real ones. As we used very little data, we need better priors to improve model accuracy.

2. Words and Topics as Features:
   Categorical features (such as words and topics) are not very common in Bayesian ML in my experience.
   Moreover, topics as features was not very useful before either in shallow models. In fact, they performed better on the raw vectors. Here, we could not use all of the features. We will see how it goes with the LDA approach in the next section.

3. Different model architectures may help improve the performance.

Another possibility: Here, we only checked the accuracy of the training. Let's see how we do on the validation and test accuracies. If the test accuracy is close to the training accuracy, then our model is actually great and was not overfitted as the models in Sections 1 and 2 were. But, as we will see, the test accuracy is even lower, showing our model is not performing so well.

In [17]:
test_X = lsa_topic_model_4b.transform(test_sentences_one_hot)
with bayes_model_4b:
    # Set the unseen test data to replace the training data
    pm.set_data({'X': test_X}) 
    # Predict response data for new data
    posterior_predictive_4b_test = pm.sample_posterior_predictive(trace_4b, predictions = 1)

Sampling: [yl]


Output()

In [18]:
y_pred_all = posterior_predictive_4b_test["predictions"]["yl"]
y_true_all = test_labels

# The first chain only
cm = [confusion_matrix(y_true = y_true_all, y_pred = y_pred_all[0][i]) for i in range(y_pred_all[0].shape[0])]
accuracy_ = [accuracy(item) for item in cm]
pd.DataFrame({'accuracy':accuracy_}).describe()

,accuracy
count,2000.000000
mean,0.240784
std,0.016586
min,0.186000
25%,0.230000
50%,0.241000
75%,0.252000
max,0.302000


As we see test accuracy is quite poor and much lower than train accuracy.

### Saving and Loading Models

In [19]:
# We need to save both the model and the trace.
# Save the model
with open(SAVE_DIR + 'bayes_model_4b.pkl', 'wb') as file_to_write:
    dpickle.dump(bayes_model_4b, file_to_write)
                                                    
# Save the trace
az.to_netcdf(trace_4b, SAVE_DIR + '/trace_model_4b.nc')

'Saved_BL_models_Exp4/trace_model_4b.nc'

In [20]:
# Load the model
with open(SAVE_DIR + 'bayes_model_4b.pkl', 'rb') as file_to_read:
    loaded_bayes_model_4b = dpickle.load(file_to_read)

# Load the trace 
loaded_trace = az.from_netcdf('Saved_BL_models_Exp4/trace_model_4b.nc') 
# Use the loaded trace for posterior predictive sampling 
with loaded_bayes_model_4b:
    posterior_predictive_new = pm.sample_posterior_predictive(loaded_trace)

Sampling: [yl]


Output()

## Model 4c: LDA Topic Model + PyMC model

### Latent Dirichlet Allocation (LDA) Model

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

num_topics = 100 # Need to be less than the total number of features

lda_topic_model_4c = LatentDirichletAllocation(n_components = num_topics, random_state = 2)  # Adjust the number of components as needed
# train_sentences_reduced = lda_topic_model_4c.fit_transform(train_sentences_one_hot)
# print(train_sentences_reduced.shape)

### PyMC Model

In [22]:
num_classes = np.unique(data.y).shape[0]
num_features = num_topics ## An important change!

# Input data

train_X = lda_topic_model_4c.fit_transform(train_sentences_one_hot)
train_y = train_labels

# Define and fit model.
with pm.Model() as bayes_model_4c:
    alpha = pm.Normal('alpha', mu=0, sigma=2, shape= num_classes)
    beta = pm.Normal('beta', mu=0, sigma=2, shape=(num_features, num_classes))

    mu = alpha + pm.math.dot(train_X, beta) # The observed input here!
    # theta = tt.nnet.softmax(mu)
    theta = pt.special.softmax(mu, axis = 1) # Softmax over the mu values
    yl = pm.Categorical('yl', p=theta, observed = train_y) # observed train_labels here
    trace_4c = pm.sample(2000)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta]


Output()

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 29 seconds.


### Predict using the Posterior Distribution

In [23]:
# Posterior predictive sampling 
posterior_predictive_4c = pm.sample_posterior_predictive(trace_4c, predictions = 1, model = bayes_model_4c)

Sampling: [yl]


Output()

### Evaluating Accuracy

In [24]:
y_pred_all = posterior_predictive_4c["predictions"]["yl"]

# The first chain only
cm = [confusion_matrix(y_true = train_labels, y_pred = y_pred_all[0][i]) for i in range(y_pred_all[0].shape[0])]

accuracy_ = [accuracy(item) for item in cm]
pd.DataFrame({'accuracy':accuracy_}).describe()

,accuracy
count,2000.000000
mean,0.384958
std,0.042156
min,0.250000
25%,0.358333
50%,0.383333
75%,0.416667
max,0.533333


As we can see, using LDA instead of LSA has worsened performance to some extent. The accuracy is down to 38% from 50%. Similar reductions can be observed in the minimum and maximum accuracies. In the future, we will do more experiments using different combinations of models, such as using graph embeddings (from Section 3) as input for the Bayesian models.